In [3]:
import re
import os
import agate

In [4]:
inputFile = os.getcwd() + '/delays.txt';
lines = open(inputFile).readlines()

In [17]:
delays = []
for line in lines:
    trim = line.strip()
    has_date = re.search(r'(\d+/\d+/\d+)', trim)
    has_time = re.search(r'(\d+:\d+)', trim)
    split = re.split(' {1,}', trim)

    if has_date and has_time and has_date.start() == 0 and len(split) > 2:
        date = split[0]
        time = split[1] + ' ' + split[2]
        datetime = date + ' ' + time
        id = split[5]
        minutes = split[6]
        delays.append((date, time, datetime, id, minutes))        

data = agate.Table(delays, column_names=['date', 'time', 'datetime', 'id', 'minutes'], column_types=[agate.Text(), agate.Text(), agate.DateTime(), agate.Text(), agate.Number()])

In [18]:
data.print_table(max_rows=5)

|---------+----------+---------------------+------+----------|
|  date   | time     |            datetime | id   | minutes  |
|---------+----------+---------------------+------+----------|
|  6/1/15 | 6:00 am  | 2015-06-01 06:00:00 | 2034 |      18  |
|  6/1/15 | 7:20 am  | 2015-06-01 07:20:00 | 1059 |      34  |
|  6/1/15 | 7:20 am  | 2015-06-01 07:20:00 | 1059 |      34  |
|  6/1/15 | 8:40 am  | 2015-06-01 08:40:00 | 010  |      15  |
|  6/1/15 | 10:39 am | 2015-06-01 10:39:00 | 2012 |      20  |
|  ...    | ...      |                 ... | ...  |     ...  |
|---------+----------+---------------------+------+----------|


In [14]:
median_delay = data.aggregate(agate.Median('minutes'))
print('The median delay was %d minutes' % median_delay)

The median delay is 9 minutes


In [19]:
total_delays = len(data.rows)
print('There were %d total delays' % total_delays)

There are 861 total delays


In [21]:
with_new_trains = data.where(lambda row: row['id'].startswith('2'))

In [22]:
total_delays_new_trains = len(with_new_trains.rows)
print('There were %d delays with new trains' % total_delays_new_trains)

There were 288 delays with new trains


In [27]:
percent_new_trains = total_delays_new_trains / total_delays * 100
print('%.2f%% of delays were with new trains' % percent_new_trains)

33.45% of delays were with new trains
